# this script is to calculate drought index from climate data
##

https://climate-indices.readthedocs.io/en/latest/



In [1]:
import sys, glob
import numpy as np
import xarray as xr
import pandas as pd
sys.path.insert(0, "climindices/src")
from climate_indices import compute, indices, utils
from climate_indices.compute import scale_values, Periodicity
#==================

In [2]:
#==================
# compute spi spei
if 0:
    ifiles = sorted(glob.glob('download/era/data/*.nc') )
    
    ds = xr.open_mfdataset(ifiles) #['tp']
    do = ds.isel(longitude = np.arange(0,360,2), latitude = np.arange(0,181,2) )
    do.to_netcdf('era5_2deg.nc')

In [3]:
tp = xr.open_dataset('era5_2deg.nc')['tp'][:,::-1]

In [4]:
# ERA5 data
dg = tp.stack(point=('latitude', 'longitude')).groupby('point')

In [9]:
#dg.values

AttributeError: 'DataArrayGroupBy' object has no attribute 'values'

In [5]:
nu_year = pd.to_datetime( tp.time.values).year.unique().size
spi_args = {
        'scale': 3,
        'distribution': indices.Distribution.gamma,
        'data_start_year': 1,
        'calibration_year_initial': 1,
        'calibration_year_final': nu_year,
        'periodicity': compute.Periodicity.monthly
}

#x = dg.values.reshape(dg.shape[0], -1)

In [8]:
da_spi = xr.apply_ufunc(
    indices.spi,
    dg,
    dask="allowed",
    kwargs=spi_args,
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name '_logger':[0m [1m[1mCannot determine Numba type of <class 'logging.Logger'>[0m
[1m
File "climindices/src/climate_indices/indices.py", line 116:[0m
[1mdef spi(
    <source elided>
        message = f"Invalid shape of input array: {shape} -- only 1-D and 2-D arrays are supported"
[1m        _logger.error(message)
[0m        [1m^[0m[0m
[0m

In [ ]:
da_spi = da_spi.unstack('point')

In [ ]:
da_spi.to_netcdf('spi_3month.nc')

In [ ]:
da_spi